<a href="https://colab.research.google.com/github/SeunghyeChae/CAKD3_tmdhey/blob/main/13.%20%EB%94%A5%EB%9F%AC%EB%8B%9D/dl_bert%EC%8B%A4%EC%8A%B5_%EB%84%A4%EC%9D%B4%EB%B2%84%EC%98%81%ED%99%94%ED%8F%89%EA%B0%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# https://huggingface.co/transformers/

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.4 MB/s 
     |████████████████████████████████| 895 kB 62.0 MB/s 
     |████████████████████████████████| 3.3 MB 59.3 MB/s 
     |████████████████████████████████| 596 kB 72.3 MB/s 
     |████████████████████████████████| 59 kB 6.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 5.1 MB/s 


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import *
import json
from tqdm import tqdm
import os

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
!git clone https://github.com/e9t/nsmc.git


Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 23.78 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Checking out files: 100% (14737/14737), done.


In [ ]:
os.listdir('nsmc')

['ratings_train.txt',
 'ratings.txt',
 'ratings_test.txt',
 'raw',
 'README.md',
 '.git',
 'synopses.json',
 'code']

In [ ]:
train= pd.read_table('nsmc/'+'ratings_train.txt')
test= pd.read_table('nsmc/'+'ratings_test.txt')
train.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [ ]:
# 버트 인풋 만들기 
# pretrain된거 tokenizer에 넣음

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
print(tokenizer.encode('보는내내 그대로 들어맞는 예측 카리스마 없는 악역'))

[101, 9356, 11018, 31605, 31605, 110589, 71568, 118913, 11018, 9576, 119281, 9786, 79940, 23811, 40364, 9520, 23160, 102]


In [ ]:
print(tokenizer.tokenize('보는내내 그대로 들어맞는 예측 카리스마 없는 악역'))
# word piece

['보', '##는', '##내', '##내', '그대로', '들어', '##맞', '##는', '예', '##측', '카', '##리스', '##마', '없는', '악', '##역']


In [ ]:
print(tokenizer.encode('전율을 일으키는 영화. 다시 보고싶은 영화',max_length=128, pad_to_max_length=True))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[101, 9665, 119183, 10622, 9641, 119185, 66815, 42428, 119, 25805, 98199, 119088, 10892, 42428, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
# 마스크 인풋
valid_num= len(tokenizer.encode('전율을 일으키는 영화. 다시 보고싶은 영화'))
print(valid_num * [1]+ (128-valid_num) * [0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
# 네이버 영화 평가 문장들을 버트 인풋으로 변환

def convert_data(data_df):
  global tokenizer

  SEQ_LEN= 128  # bert 인풋의 길이 

  tokens, masks, segments, targets = [],[],[],[]

  for i in tqdm(range(len(data_df))):
    token = tokenizer.encode(data_df[DATA_COLUMN][i], max_length= SEQ_LEN, truncation=True,padding='max_length')

    num_zeros= token.count(0)
    mask = [1]* (SEQ_LEN-num_zeros) + [0]*num_zeros

    segment = [0] *SEQ_LEN

    tokens.append(token)
    masks.append(mask)
    segments.append(segment)

    targets.append(data_df[LABEL_COLUMN][i])

  tokens= np.array(tokens)
  masks= np.array(masks)
  segments= np.array(segments)
  targets= np.array(targets)

  return  [tokens,masks, segments], targets

def load_data(pandas_dataframe):
  data_df = pandas_dataframe
  data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
  data_df[LABEL_COLUMN] = data_df[LABEL_COLUMN].astype(int)
  data_x, data_y = convert_data(data_df)
  return data_x, data_y


SEQ_LEN= 128
BATCH_SIZE = 20
DATA_COLUMN = 'document'
LABEL_COLUMN= 'label'

train_x, train_y= load_data(train)
# test_x, test_y= load_data(test)

100%|██████████| 150000/150000 [01:00<00:00, 2464.75it/s]


In [ ]:
train_x

# 토큰
# 마스크
# 세그먼트

[array([[  101,  9519,  9074, ...,     0,     0,     0],
        [  101,   100,   119, ...,     0,     0,     0],
        [  101,   100,   102, ...,     0,     0,     0],
        ...,
        [  101,  9638, 14153, ...,     0,     0,     0],
        [  101,  9751, 97707, ...,     0,     0,     0],
        [  101, 48556, 42428, ...,     0,     0,     0]]),
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

In [ ]:
test_x, test_y= load_data(test)

100%|██████████| 50000/50000 [00:19<00:00, 2554.32it/s]


In [ ]:
# 버트를 활용한 감성분석 모델 만들기 

TPU = True
if TPU:
  resolver= tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://'+ os.environ['COLAB_TPU_ADDR'])
  tf.config.experimental_connect_to_cluster(resolver)
  tf.tpu.experimental.initialize_tpu_system(resolver)
else:
  pass



INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.107.188.186:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.107.188.186:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


In [ ]:
# 버트 베이스 레이어:12 
# 라지 레이어: 24 


In [ ]:
# Rectified Adam 옵티마이저 사용 

!pip install tensorflow_addons
import tensorflow_addons as tfa
opt= tfa.optimizers.RectifiedAdam(learning_rate=1.0e-5, weight_decay=0.0025,warmup_proportion= 0.05)
# weight_decay : weight가 너무 큰 값을 갖지 않도록 조정

     |████████████████████████████████| 1.1 MB 5.3 MB/s 


In [ ]:
# 사용자 함수로 버트 모델 만들기 
# 감정 분석에 맞는 버트 모형 리턴하는 함수 정의 
def create_sentiment_bert():
  # 버트 pretrained 모델 로드
  model = TFBertModel.from_pretrained('bert-base-multilingual-cased')
  # 토큰 인풋, 마스크 인풋, 세그먼트 인풋 정의
  token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids')
  mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks')
  segment_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_segment')
  # 인풋이 [토큰, 마스크, 세그먼트]인 모델 정의
  bert_outputs = model([token_inputs, mask_inputs, segment_inputs])

  bert_outputs = bert_outputs[1]
  sentiment_first = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))(bert_outputs)
  sentiment_model = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], sentiment_first)

  sentiment_model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics = ['accuracy'])
  return sentiment_model

In [ ]:
# TPU 실행 시
if TPU:
  strategy = tf.distribute.experimental.TPUStrategy(resolver)
# 함수를 strategy.scope로 묶어 줌
  with strategy.scope():
    sentiment_model = create_sentiment_bert()
  
  sentiment_model.fit(train_x, train_y, epochs=4, shuffle=True, batch_size=100, validation_data=(test_x, test_y))
else:
  # GPU 모드로 훈련시킬 때
  sentiment_model = create_sentiment_bert()
  
  sentiment_model.fit(train_x, train_y, epochs=4, shuffle=True, batch_size=100, validation_data=(test_x, test_y))

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

Epoch 1/4


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'IteratorGetNext:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'IteratorGetNext:1' shape=(None, 128) dtype=int64>, <tf.Tensor 'IteratorGetNext:2' shape=(None, 128) dtype=int64>, <tf.Tensor 'IteratorGetNext:3' shape=(None,) dtype=int64>]
INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'IteratorGetNext:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'IteratorGetNext:1' shape=(None, 128) dtype=int64>, <tf.Tensor 'IteratorGetNext:2' shape=(None, 128) dtype=int64>, <tf.Tensor 'IteratorGetNext:3' shape=(None,) dtype=int64>]


1500/1500 [==============================] - ETA: 0s - loss: 0.4485 - accuracy: 0.7801

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'IteratorGetNext:0' shape=(None, 128) dtype=int64>, <tf.Tensor 'IteratorGetNext:1' shape=(None, 128) dtype=int64>, <tf.Tensor 'IteratorGetNext:2' shape=(None, 128) dtype=int64>, <tf.Tensor 'IteratorGetNext:3' shape=(None,) dtype=int64>]


1500/1500 [==============================] - 317s 145ms/step - loss: 0.4485 - accuracy: 0.7801 - val_loss: 0.3640 - val_accuracy: 0.8381
Epoch 2/4
1500/1500 [==============================] - 207s 138ms/step - loss: 0.3347 - accuracy: 0.8505 - val_loss: 0.3261 - val_accuracy: 0.8573
Epoch 3/4
1500/1500 [==============================] - 207s 138ms/step - loss: 0.2942 - accuracy: 0.8722 - val_loss: 0.3168 - val_accuracy: 0.8616
Epoch 4/4
1500/1500 [==============================] - 207s 138ms/step - loss: 0.2592 - accuracy: 0.8898 - val_loss: 0.3098 - val_accuracy: 0.8712


https://github.com/kimwoonggon/publicservant_AI/blob/master/1_(HuggingFace)%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%98%81%ED%99%94_%ED%8F%89%EA%B0%80_%EA%B8%8D%EB%B6%80%EC%A0%95_%EB%B6%84%EC%84%9D.ipynb